In [1]:
from scipy.sparse import csr_array

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
ACTIVITY_TRAIN = pd.read_csv('../activity_train.csv', names=['uniprot_id', 'mol_id', 'activity'])
ACTIVITY_TRAIN

,uniprot_id,mol_id,activity
0,O14842,CHEMBL2022243,4
1,O14842,CHEMBL2022244,6
2,O14842,CHEMBL2022245,2
3,O14842,CHEMBL2022246,1
4,O14842,CHEMBL2022247,4
...,...,...,...
135706,Q9Y5Y4,CHEMBL4214909,6
135707,Q9Y5Y4,CHEMBL4218012,2
135708,Q9Y5Y4,CHEMBL4217503,7
135709,Q9Y5Y4,CHEMBL4204359,8


Maybe we should have a matrix like...

```
    , unitprot1 , unitprot2, unitprot3, unitprot4, ...
mol1,     1     ,     5    ,          ,     10    , ...
mol2,           ,     1    ,     6     ,     1    , ...
...

```

In [3]:
ACTIVITY_TRAIN

,uniprot_id,mol_id,activity
0,O14842,CHEMBL2022243,4
1,O14842,CHEMBL2022244,6
2,O14842,CHEMBL2022245,2
3,O14842,CHEMBL2022246,1
4,O14842,CHEMBL2022247,4
...,...,...,...
135706,Q9Y5Y4,CHEMBL4214909,6
135707,Q9Y5Y4,CHEMBL4218012,2
135708,Q9Y5Y4,CHEMBL4217503,7
135709,Q9Y5Y4,CHEMBL4204359,8


In [4]:
def CosSim_Matrix(M):
    norms=np.sqrt(np.sum(M*M, axis=1))
    norms[norms<0.001]=0.001  #this will solve rows or cols without variance
    norms_M = np.outer(norms, norms) # ESTA LINHA FAZ CRASHAR O PC
    VC=M.copy()
    return np.dot(VC, VC.T)/norms_M

def RowCenterMatrix(M):
    V  = M.T
    mat= np.nanmean(V, axis=0)
    VC = V - mat
    VC[np.isnan(VC)]=0
    return VC.T

In [5]:
def create_rankingMatrix(rowLabel, colLabel, df):
    rows = list(set(ACTIVITY_TRAIN[rowLabel])) 
    cols = list(set(ACTIVITY_TRAIN[colLabel])) 
    n_rows = len(rows)
    n_cols = len(cols)

    rows = dict(zip(rows, np.arange(n_rows)))
    cols = dict(zip(cols, np.arange(n_cols)))
    mat = np.zeros((n_rows, n_cols))
    mat[mat==0]=np.nan
    for rw in df.values:
        mat[rows[rw[0]], cols[rw[1]]]=rw[2]
    return mat

mat=create_rankingMatrix("uniprot_id", "mol_id", ACTIVITY_TRAIN)
mat=mat.T
pd.DataFrame(mat)

,0,1,2,3,4,5,6,7,8,9,...,134,135,136,137,138,139,140,141,142,143
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN


In [ ]:
def estimate_score(df, SM, nn, r,c, verbose=False):
    vals=df.values
    N,M=vals.shape
    sims=list(zip(SM[r], range(N)))
    sims.sort()
    sims.reverse()
    cnt=0
    S=0
    Ssims=0
    if verbose: print("row: %d - col: %d" % (r,c))
    for  sim, idx in sims[1:]:
        if not np.isnan(vals[idx, c]) and sim>0:
            cnt   += 1
            S     += sim*vals[idx, c]
            Ssims += sim
            if verbose: print("\tItem:%4d    Score:%4.1f (Sim: %6.3f)" %(idx, vals[idx, c], sim))
        if cnt>= nn: break
    #this is the situation where no nighbours were found
    if Ssims<=0:
        #this is very inneficient!!! <- better have it precomputed 
        if verbose: print("\tNo similars: outputing the Global Baseline Average")
        M = np.nanmean(df.values)
        rA= np.nanmean(df.values[r])
        cA= np.nanmean(df.values[:,c])
        r = min(max(rA+cA-M,0),5)
        if verbose: print("\tScore: %7.4f" % r)
        return r
    if verbose: print("\tScore: %7.4f" % (S/Ssims))
    return S/Ssims

In [6]:
mat_centered=RowCenterMatrix(mat)

In [7]:
mat_centered

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
csr_mat = csr_array(mat_centered)

In [9]:
#sim_mat = CosSim_Matrix(csr_mat) # ISTO CRASHA O PC
sim_mat = CosSim_Matrix(mat_centered)

MemoryError: Unable to allocate 39.3 GiB for an array with shape (72632, 72632) and data type float64